In [22]:
import os as os
import nltk as nltk
import numpy as np
import re
import requests
from sklearn.datasets import load_svmlight_files
from sklearn.naive_bayes import GaussianNB
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
rootDirectory = "aclImdb/"
trainPos = "train/pos/"
trainNeg = "train/neg/"
testPos = "test/pos/"
testNeg = "test/neg/"
def readDataFromFile(path):
    rList = np.array([], dtype='object')
    for file in os.listdir(path):
        rList = np.append(rList, open(path + file, encoding="utf8").read()) 
    return rList


In [24]:
# prepare the data set for problem 1 and 2
positiveTrainReviwList = readDataFromFile(rootDirectory + trainPos)
negativeTrainReviwList = readDataFromFile(rootDirectory + trainNeg)
positiveTestReviwList = readDataFromFile(rootDirectory + testPos)
negativeTestReviwList = readDataFromFile(rootDirectory + testNeg)
xTrain = np.concatenate((positiveTrainReviwList, negativeTrainReviwList), axis = None)
yTrain = np.concatenate((np.ones(len(positiveTrainReviwList)), np.zeros(len(negativeTrainReviwList))), axis = None)
xTest = np.concatenate((positiveTestReviwList, negativeTestReviwList), axis = None)
yTest = np.concatenate((np.ones(len(positiveTestReviwList)), np.zeros(len(negativeTestReviwList))), axis = None)

## Problem 1

In [31]:
# Given the level of testing data and level of predicted data, this method will return true positive, 
# true negative, false positive and false negative
def calcaulateConfusionMatrix(testData, predictedData):
    i = 0
    tp = fp = tn = fn = 0
    for p in  predictedData:
        if p == 1:
            if p == testData[i]:
                tp += 1
            else:
                fp += 1
        else:
            if p == testData[i]:
                tn += 1
            else:
                fn += 1
        i += 1
    return tp, tn, fp, fn

In [32]:
# Given the level of testing data and level of predicted data, this will calcualte Precision using confusion matrix
def calculatePrecision(yTrue, yPredicted):
    tp, tn, fp, fn = calcaulateConfusionMatrix(yTrue, yPredicted)
    return tp / (tp + fp)

In [33]:
# Given the level of testing data and level of predicted data, this will calcualte Recall using confusion matrix
def calculateRecall(yTrue, yPredicted):
    tp, tn, fp, fn = calcaulateConfusionMatrix(yTrue, yPredicted)
    return tp / (tp + fn)

In [34]:
# Given the level of testing data and level of predicted data, this will calcualte fScore using confusion matrix
def calculateFscore(yTrue, yPredicted):
    tp, tn, fp, fn = calcaulateConfusionMatrix(yTrue, yPredicted)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return (2 * precision * recall) / (precision + recall)

## Problem 2

In [35]:
# A classifier which always output the level of majority class as predicted label 
def majorityClassBaselineClassifier(xTrain, yTrain, xTest):
    if np.count_nonzero(yTrain) >= (len(yTrain) - np.count_nonzero(yTrain)):
        return np.ones(len(xTest))
    return np.zeros(len(xTest))

In [37]:
#predict the label using Training data as Test data
prediction = majorityClassBaselineClassifier(xTrain, yTrain, xTrain)
print("P, R, F for trainData::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTrain, prediction)
                                                                          , calculateRecall(yTrain, prediction)
                                                                        , calculateFscore(yTrain, prediction)))
#predict the label using Training data as Test data  
prediction = majorityClassBaselineClassifier(xTrain, yTrain, xTest)
print("P, R, F for testData::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTest, prediction)
                                                                          , calculateRecall(yTest, prediction)
                                                                          , calculateFscore(yTest, prediction)))

P, R, F for trainData::: Precision: 0.500000, Recall: 1.000000, and F-score: 0.666667 

P, R, F for testData::: Precision: 0.500000, Recall: 1.000000, and F-score: 0.666667 



## Problem 3

In [38]:
# A classifier which predicts the label based on the review length
def reviewLengthBaselineClassifier(x_train, x_test, threshold):
    prediction = np.array([], dtype='object')
    for x in x_test:
        if len(x) > threshold:
            prediction = np.append(prediction, 1)
        else:
            prediction = np.append(prediction, 0)       
    return prediction

In [46]:
prediction = reviewLengthBaselineClassifier(xTrain, xTrain, 60)
print("P, R, F for trainData with threshold 60::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTrain, prediction)
                                                                          , calculateRecall(yTrain, prediction)
                                                                          , calculateFscore(yTrain, prediction)))
prediction = reviewLengthBaselineClassifier(xTrain, xTest, 60)
print("P, R, F for testData with threshold 60::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTest, prediction)
                                                                          , calculateRecall(yTest, prediction)
                                                                          , calculateFscore(yTest, prediction)))
prediction = reviewLengthBaselineClassifier(xTrain, xTrain, 80)
print("P, R, F for trainData with threshold 80::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTrain, prediction)
                                                                          , calculateRecall(yTrain, prediction)
                                                                          , calculateFscore(yTrain, prediction)))
prediction = reviewLengthBaselineClassifier(xTrain, xTest, 80)
print("P, R, F for testData with threshold 80::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTest, prediction)
                                                                          , calculateRecall(yTest, prediction)
                                                                          , calculateFscore(yTest, prediction)))
prediction = reviewLengthBaselineClassifier(xTrain, xTrain, 400)
print("P, R, F for trainData with threshold 400::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTrain, prediction)
                                                                          , calculateRecall(yTrain, prediction)
                                                                          , calculateFscore(yTrain, prediction)))
prediction = reviewLengthBaselineClassifier(xTrain, xTest, 400)
print("P, R, F for testData with threshold 400::: Precision: %f, Recall: %f, and F-score: %f \n"%(calculatePrecision(yTest, prediction)
                                                                          , calculateRecall(yTest, prediction)
                                                                          , calculateFscore(yTest, prediction)))

P, R, F for trainData with threshold 60::: Precision: 0.500040, Recall: 1.000000, and F-score: 0.666702 

P, R, F for testData with threshold 60::: Precision: 0.500080, Recall: 1.000000, and F-score: 0.666738 

P, R, F for trainData with threshold 80::: Precision: 0.500080, Recall: 0.999920, and F-score: 0.666720 

P, R, F for testData with threshold 80::: Precision: 0.500060, Recall: 0.999840, and F-score: 0.666684 

P, R, F for trainData with threshold 400::: Precision: 0.494995, Recall: 0.925760, and F-score: 0.645075 

P, R, F for testData with threshold 400::: Precision: 0.496517, Recall: 0.923760, and F-score: 0.645878 



## Problem 4

In [40]:
# label will be generated based on the rating present in labeledBow.feat file
def generateLabel(data):
    label = []
    for d in data:
        if d > 5:
            label.append(1)
        else:
            label.append(0)
    return label

In [41]:
#### Naive bayes classifier-GaussianNB with features from labeledBow.feat file

bowFilePath = ["aclImdb/train/labeledBow.feat","aclImdb/test/labeledBow.feat"]
xTraining, yTraining, xTesting, yTesting = load_svmlight_files(bowFilePath, n_features=None, dtype=None)
xTraining = np.int16(xTraining.todense())
xTesting = np.int16(xTesting.todense())
labelTraing = generateLabel(yTraining)
labelTesting = generateLabel(yTesting)
naiveBayesModel = GaussianNB().fit(xTraining, labelTraing)

predicted= naiveBayesModel.predict(xTraining)
print("P, R, F for training Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTraing, predicted), calculateRecall(labelTraing, predicted)
        , calculateFscore(labelTraing, predicted)))

predicted= naiveBayesModel.predict(xTesting)
print("P, R, F for testing Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTesting, predicted), calculateRecall(labelTesting, predicted)
        , calculateFscore(labelTesting, predicted)))

P, R, F for training Data::: Precision: 0.992216, Recall: 0.846400, and F-score: 0.913526 

P, R, F for testing Data::: Precision: 0.624640, Recall: 0.433200, and F-score: 0.511597 



In [ ]:
#### Naive bayes classifier-GaussianNB with features from labeledBow.feat file 
### and at the end transforming it with TfidfTransformer

tf_transformer = TfidfTransformer()
xTraining1 = tf_transformer.fit_transform(xTraining)
xTesting1 = tf_transformer.fit_transform(xTesting)

naiveBayesModel1 = GaussianNB().fit(xTraining1.toarray(), labelTraing)

predicted= naiveBayesModel1.predict(xTraining1.toarray())
print("P, R, F for training Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTraing, predicted), calculateRecall(labelTraing, predicted)
        , calculateFscore(labelTraing, predicted)))

predicted= naiveBayesModel1.predict(xTesting1.toarray())
print("P, R, F for testing Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTesting, predicted), calculateRecall(labelTesting, predicted)
        , calculateFscore(labelTesting, predicted)))


In [61]:
##### naive bayes classifier-GaussianNB with features calculation using TfidfVectorizer

trainingData = np.concatenate((positiveTrainReviwList, negativeTrainReviwList), axis=None)
testingData = np.concatenate((positiveTrainReviwList, negativeTestReviwList), axis=None)


vectorizer = TfidfVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=100)

trainingData = vectorizer.fit_transform(trainingData)
testingData = vectorizer.fit_transform(testingData)

naiveBayesModel2 = GaussianNB().fit(trainingData.toarray(), yTrain)

predicted= naiveBayesModel2.predict(trainingData.toarray())
print("P, R, F for training Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(yTrain, predicted), calculateRecall(yTrain, predicted)
        , calculateFscore(yTrain, predicted)))

predicted= naiveBayesModel2.predict(testingData.toarray())
print("P, R, F for testing Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(yTest, predicted), calculateRecall(yTest, predicted)
        , calculateFscore(yTest, predicted)))


P, R, F for training Data::: Precision: 0.665659, Recall: 0.704960, and F-score: 0.684746 

P, R, F for testing Data::: Precision: 0.664780, Recall: 0.704560, and F-score: 0.684092 



## Problem 5

In [64]:
#### LogisticRegression with features from labeledBow.feat file and at the end transforming it with TfidfTransformer
## xTraining and xTesting are calculated by extrating features from from labeledBow.feat file 
## and  transforming it with TfidfTransformer

lRModel = LogisticRegression().fit(xTraining.toarray(), labelTraing)

predicted= lRModel.predict(xTraining.toarray())
print("P, R, F for training Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTraing, predicted), calculateRecall(labelTraing, predicted)
        , calculateFscore(labelTraing, predicted)))

predicted= lRModel.predict(xTesting.toarray())
print("P, R, F for testing Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTesting, predicted), calculateRecall(labelTesting, predicted)
        , calculateFscore(labelTesting, predicted)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


P, R, F for training Data::: Precision: 0.926899, Recall: 0.937280, and F-score: 0.932060 

P, R, F for testing Data::: Precision: 0.881565, Recall: 0.884880, and F-score: 0.883220 



In [62]:
#### LogisticRegression with features calculation using TfidfVectorizer
## trainingData and testingData are calculated by extrating features using TfidfVectorizer

lRModel1 = LogisticRegression().fit(trainingData.toarray(), labelTraing)

predicted= lRModel1.predict(trainingData.toarray())
print("P, R, F for training Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTraing, predicted), calculateRecall(labelTraing, predicted)
        , calculateFscore(labelTraing, predicted)))

predicted= lRModel1.predict(testingData.toarray())
print("P, R, F for testing Data::: Precision: %f, Recall: %f, and F-score: %f \n"
      %(calculatePrecision(labelTesting, predicted), calculateRecall(labelTesting, predicted)
        , calculateFscore(labelTesting, predicted)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


P, R, F for training Data::: Precision: 0.708044, Recall: 0.726000, and F-score: 0.716910 

P, R, F for testing Data::: Precision: 0.701471, Recall: 0.725040, and F-score: 0.713061 



## Problem 6

## Problem 2
:::trainData::: Precision: 0.500000, Recall: 1.000000, and F-score: 0.666667 <br>
:::testData::: Precision: 0.500000, Recall: 1.000000, and F-score: 0.666667 
since number of positive and negative reviews are equal for both test data and train data, the precision is reasonable.

## Problem 3
:::trainData with threshold 60::: Precision: 0.500040, Recall: 1.000000, and F-score: 0.666702 <br>
:::testData with threshold 60::: Precision: 0.500080, Recall: 1.000000, and F-score: 0.666738 <br>

:::trainData with threshold 80::: Precision: 0.500080, Recall: 0.999920, and F-score: 0.666720 <br>
:::testData with threshold 80::: Precision: 0.500060, Recall: 0.999840, and F-score: 0.666684 <br>

:::trainData with threshold 400::: Precision: 0.494995, Recall: 0.925760, and F-score: 0.645075 <br>
:::testData with threshold 400::: Precision: 0.496517, Recall: 0.923760, and F-score: 0.645878 

<br>From the result, it seems that all of the positive review length are less than 400. and negative review length are greater than or equal 400.

## Problem 4
#### Naive bayes classifier-GaussianNB with features from labeledBow.feat file
:::training Data::: Precision: 0.992216, Recall: 0.846400, and F-score: 0.913526  
:::testing Data::: Precision: 0.624640, Recall: 0.433200, and F-score: 0.511597 

#### Naive bayes classifier-GaussianNB with features from labeledBow.feat file and at the end transforming it with TfidfTransformer
:::training Data::: Precision: 0.993969, Recall: 0.949360, and F-score: 0.971153 <br> 
:::testing Data::: Precision: 0.608667, Recall: 0.521360, and F-score: 0.561641

#### Naive bayes classifier-GaussianNB with features calculation using TfidfVectorizer
:::training Data::: Precision: 0.665659, Recall: 0.704960, and F-score: 0.684746 <br> 
:::testing Data::: Precision: 0.664780, Recall: 0.704560, and F-score: 0.684092 

<br>From the analysis of above presented results of 3 cases for GaussianNB classifier, it is clear that the classifier gives better results when features are calculated using already calculated BOW feature file provided with dataset- labeledBow.feat and transforming it to TFIDF features.
<br>From the reported result, GaussianNB classifier with TfidfVectorizer gives worst result if stop words are removed, strips_accent ='unicode' and  ngram range is (1,3), and gives best result when the features are used from labeledBow.feat file.
Though using TfidfVectorizer gives best result for test data, the precision of traning data drops from 99% to 69%. One reason for this can be the removal of stop word and accent may remove some of the impotant freatures of reviews. At th end i removed the stop word removing, accent removing and set to use unigram model for TfidfVectorizer. In this case, the precision improves a little, however still not much to consider.

## Problem 5
#### LogisticRegression with features from labeledBow.feat file and at the end transforming it with TfidfTransformer
:::training Data::: Precision: 0.926899, Recall: 0.937280, and F-score: 0.932060 <br> 
:::testing Data::: Precision: 0.881565, Recall: 0.884880, and F-score: 0.883220 

#### LogisticRegression with features calculation using TfidfVectorizer
:::training Data::: Precision: 0.708044, Recall: 0.726000, and F-score: 0.716910<br> 
:::testing Data::: Precision: 0.701471, Recall: 0.725040, and F-score: 0.713061 

<br> Like the problem 4 the logistic regression gives best precision if the features are used from labeledBow.feat file and transformed it TFIDF features.
 
<br> For the 2 cases of Logistic Regression, it also gives better results when features are calculated using already calculated BOW feature file provided with dataset- labeledBow.feat and transforming it to TFIDF features.<br> 
 Among the 2 classifier logistic regression classifier gives much better results with accuracy of around 88% and recall of 88.5%

## Problem 7

In [47]:
## Load the word2vec pretrained model from google news dataset
filename = 'GoogleNews-vectors-negative300.bin'
googleModel = KeyedVectors.load_word2vec_format(filename, binary=True, limit=500000)

In [48]:
## Load the globe word2vec pretrained model
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
filename = 'glove.6B.100d.txt.word2vec'
gloveModel = KeyedVectors.load_word2vec_format(filename, binary=False)

In [53]:
#find out top word for a analogy test for all given category
def testAanalogy(modelName, model, analogySet):
    groupAccuracy = {}
    highestSimilarWord = {}
    for key in analogySet:
        TP = 0
        failed = 0
        for analogy in analogySet[key]:
            if all(word in model for word in analogy):
                highestSimilarWord = model.most_similar(positive = [analogy[1], analogy[2]]
                                                        , negative = [analogy[0]], topn=1)
                print(highestSimilarWord[0][0])
                if highestSimilarWord[0][0] == analogy[3]:
                    TP += 1
            else:
                # Failed prediction because of vocabulary absence
                if failed == (len(analogySet[key]) - 1):
                    print("One/more Vocabulary missing for this analogy: ", analogy)
                failed += 1

        
        #print("Category:", key, ", Failed:", failed, ", Total:", len(analogySet[key]), (len(analogySet[key]) - failed), ", TP:", TP)              
        if len(analogySet[key]) == failed:
            groupAccuracy[key] = 0
        else:
            groupAccuracy[key] = TP / (len(analogySet[key]) - failed)
    print(modelName, ":\n", groupAccuracy, "\n")

## Problem 7 : Reporting the Accuracy

In [50]:
##Read Mikelov data set and preprocess it
analogyDataSet = requests.get("http://www.fit.vutbr.cz/~imikolov/rnnlm/word-test.v1.txt")
analogyDataSet = analogyDataSet.text.split("\n")

categoryDict = {}
for item in analogyDataSet[1:]:
    if ":" in item:
        cKey = item.strip().lower()
        categoryDict[cKey] = []
    else:
        categoryDict[cKey].append(item.strip().lower().split())   
        
requiredGroupList = [': capital-world', ': currency', ': city-in-state', ': family',
': gram1-adjective-to-adverb', ': gram2-opposite', ': gram3-comparative', ': gram6-nationality-adjective']
requiredGroupAnalogy = {}
for rGroup in requiredGroupList:
    requiredGroupAnalogy[rGroup] = categoryDict[rGroup]

# Report the accuracy of the mentioned analogy 
testAanalogy("Glove pretrained model on Wikipedia", gloveModel, requiredGroupAnalogy)
testAanalogy("Google Word2Vec 300d news data set", googleModel, requiredGroupAnalogy)

Glove pretrained model on Wikipedia :
 {': capital-world': 0.8894783377541998, ': currency': 0.1420323325635104, ': city-in-state': 0.3080664775030401, ': family': 0.8162055335968379, ': gram1-adjective-to-adverb': 0.2439516129032258, ': gram2-opposite': 0.20073891625615764, ': gram3-comparative': 0.7912912912912913, ': gram6-nationality-adjective': 0.8786741713570981} 

Google Word2Vec 300d news data set :
 {': capital-world': 0.2, ': currency': 0.17222222222222222, ': city-in-state': 0.16326530612244897, ': family': 0.8517786561264822, ': gram1-adjective-to-adverb': 0.2923387096774194, ': gram2-opposite': 0.42980295566502463, ': gram3-comparative': 0.9114114114114115, ': gram6-nationality-adjective': 0.5775862068965517} 



In this task, I used word2Vec pretrained model provied by google (googel news 300d)[2] and Glove pretrained model with 6B token 100d dataset[1]. For the required. to Run the program, one have to put the  'GoogleNews-vectors-negative300.bin' and 'glove.6B.100d.txt' pretrianed model in the same golder where this ipython notebook resides.

The accurracy for each group is reported in the above execution. Analysis the analogy accuracy reported by both glove and word2Vec, the glove word embedding out performs world2vec in the analogy prediction task. Moreover, word2vec model pretrained model built on google news data does not contain all of the vocabulary exist int he mikhelov analogy set. One reason for missing vocabulary is that i load only 500k frequently used word vector for google news word embeding pretrained model. For the accuracy calculation, if a analogy prediction task failed for missing vocabulary, i did not consider this anlogy entry in the total prediction count.

For analogy category {capital-world, city-in-state, gram6-nationality-adjective} glove has high accuracy whereas, for
analogy category {currency, family, gram1-adjective-to-adverb, gram2-opposite, gram3-comparative} word2Vec embedding has high accuracy. The analysis reveals that the word2Vec outperforms glove in most of the mikelov analogy test(except one- gram6-nationality-adjective) cases if the vocabulary is present although i used only the 500k frequently used vector for word2Vec.

1. Glove pretrained model, 6B 100d, https://nlp.stanford.edu/projects/glove/
2. Google news word2vec model, https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

## Problem 8

In [51]:
# Find out top 10 similar word of 'increase', 'hatred' and 'older'
print(gloveModel.most_similar("increase", topn=10), "\n")
print(gloveModel.most_similar("hatred", topn=10), "\n")
print(gloveModel.most_similar("older", topn=10), "\n")
print(googleModel.most_similar("increase", topn=10), "\n")
print(googleModel.most_similar("hatred", topn=10), "\n")
print(googleModel.most_similar("older", topn=10), "\n")

[('increased', 0.8988391160964966), ('increases', 0.8812485933303833), ('decrease', 0.8812355995178223), ('increasing', 0.8467099070549011), ('reduce', 0.8099488019943237), ('rise', 0.7936252951622009), ('reduced', 0.7800052165985107), ('growth', 0.7789974212646484), ('reduction', 0.7766253352165222), ('boost', 0.7759016752243042)] 

[('hostility', 0.7843601703643799), ('bigotry', 0.7682587504386902), ('intolerance', 0.7667320966720581), ('animosity', 0.756497859954834), ('resentment', 0.7466510534286499), ('enmity', 0.7424829006195068), ('bitterness', 0.7228323817253113), ('prejudice', 0.7177624106407166), ('anger', 0.7149417400360107), ('fanaticism', 0.7136397957801819)] 

[('younger', 0.828304648399353), ('age', 0.7673466205596924), ('newer', 0.688201904296875), ('ages', 0.6771453022956848), ('those', 0.6731763482093811), ('siblings', 0.670750081539154), ('parents', 0.6635492444038391), ('children', 0.6634655594825745), ('ones', 0.6595342755317688), ('young', 0.6554997563362122)] 



In the above executed program, the top 10 similar words of "increase" and "older" contains their antonym. However, the top 10 similar words of  "hatred" does not contain any anotonyms. One reason for a antonym's similarty score to be higher is that both the word and it's antonym can be used in same context in a sentence. For example, "older boy" or "younger boy", in this two example both older and younger are associated with boy as an adjective. In another word, the use of paradigmetic relation in word2vec makes it to have high similarity score of an antonym of a word becasue both of them can be used in the same context

## Problem 9

In [57]:
#Designing three new analogy and predict it using Glove and Word2vec word embedding
analogy1_performer_action = {}
analogy2_object_color = {}
analogy3_object_organ =  {}
analogy1_performer_action["performar_action"] = [["teacher", "teach", "player", "play"],
                                ["footballer", "play", "artist", "paint"],
                                ["helpe", "help", "singer", "sing"]]

analogy2_object_color["object_color"] = [['tomato', 'red', 'cabbage', 'green'],
                            ["orange", "yellow", "strawberry", "red"],
                            ["lemon", "yellow", "lime", "green"]]
analogy3_object_organ["object_organ"] = [['human', 'hand', 'plant', 'leaf'],
                            ["human", "hair", "computer", "display"],
                            ["keyboard", "key", "car", "tire"]]

testAanalogy("Glove pretrained model on Wikipedia", gloveModel, analogy1_performer_action)
testAanalogy("Glove pretrained model on Wikipedia", gloveModel, analogy2_object_color)
testAanalogy("Glove pretrained model on Wikipedia", gloveModel, analogy3_object_organ)

testAanalogy("Google Word2Vec 300d news data set", googleModel, analogy1_performer_action)
testAanalogy("Google Word2Vec 300d news data set", googleModel, analogy2_object_color)
testAanalogy("Google Word2Vec 300d news data set", googleModel, analogy3_object_organ)

players
work
and
Glove pretrained model on Wikipedia :
 {'performar_action': 0.0} 

blue
darryl
red
Glove pretrained model on Wikipedia :
 {'object_color': 0.0} 

factory
wig
vehicle
Glove pretrained model on Wikipedia :
 {'object_organ': 0.0} 

players
artists
Google Word2Vec 300d news data set :
 {'performar_action': 0.0} 

yellow
strawberries
red
Google Word2Vec 300d news data set :
 {'object_color': 0.0} 

plants
tresses
crucial
Google Word2Vec 300d news data set :
 {'object_organ': 0.0} 



In the above execution of three analogy categories, both the glove and word2Vec provide zero accuracy


## Requirement to Run the File
The aclIMDB dataset, glove.6B.100d.txt and GoogleNews-vectors-negative300.bin needs to be in the same folder as of the jupyter notebook file.